In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from pylab import rcParams
from pyspark.sql.functions import avg
import pyspark.sql.functions as f
from pyspark.sql.window import Window #***
from pyspark.sql.functions import sum
from pyspark.sql.functions import format_number


df = spark.read.csv('/home/giotanna/Documents/DM_project_2021/healthcare-dataset-stroke-data/healthcare-dataset-stroke-data.csv', inferSchema=True,header=True)

In [2]:
df.show()


+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

In [3]:
df.describe().show()

+-------+-----------------+------+------------------+------------------+-------------------+------------+---------+--------------+------------------+------------------+--------------+-------------------+
|summary|               id|gender|               age|      hypertension|      heart_disease|ever_married|work_type|Residence_type| avg_glucose_level|               bmi|smoking_status|             stroke|
+-------+-----------------+------+------------------+------------------+-------------------+------------+---------+--------------+------------------+------------------+--------------+-------------------+
|  count|             5110|  5110|              5110|              5110|               5110|        5110|     5110|          5110|              5110|              5110|          5110|               5110|
|   mean|36517.82935420744|  null|43.226614481409015|0.0974559686888454|0.05401174168297456|        null|     null|          null|106.14767710371804|28.893236911794673|          null| 

In [4]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: string (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [5]:
df.groupBy('gender').mean().show()

+------+-----------------+-----------------+-------------------+-------------------+----------------------+--------------------+
|gender|          avg(id)|         avg(age)|  avg(hypertension)| avg(heart_disease)|avg(avg_glucose_level)|         avg(stroke)|
+------+-----------------+-----------------+-------------------+-------------------+----------------------+--------------------+
|Female|36479.68503674015|43.75739478957918|0.09218436873747494|0.03774215096860387|    104.05780895123563|0.047094188376753505|
| Other|          56156.0|             26.0|                0.0|                0.0|                143.33|                 0.0|
|  Male|36562.54137115839|42.48338534278962| 0.1049645390070922|0.07706855791962175|     109.0885200945626| 0.05106382978723404|
+------+-----------------+-----------------+-------------------+-------------------+----------------------+--------------------+



In [6]:
display(df.groupBy("gender").avg("stroke").orderBy("gender"))

DataFrame[gender: string, avg(stroke): double]

In [7]:
   df.groupBy('gender','stroke').mean().show()

+------+------+------------------+-----------------+-------------------+-------------------+----------------------+-----------+
|gender|stroke|           avg(id)|         avg(age)|  avg(hypertension)| avg(heart_disease)|avg(avg_glucose_level)|avg(stroke)|
+------+------+------------------+-----------------+-------------------+-------------------+----------------------+-----------+
|  Male|     1| 37566.67592592593|             68.5|               0.25|0.25925925925925924|    143.15935185185185|        1.0|
| Other|     0|           56156.0|             26.0|                0.0|                0.0|                143.33|        0.0|
|Female|     0| 36465.37889940414|42.60193480546795|0.08307045215562565|0.03294777427269541|    103.05175254118453|        0.0|
|  Male|     0|  36508.5072247135|41.08338814150475|0.09715994020926756|0.06726457399103139|    107.25511210762329|        0.0|
|Female|     1|36769.156028368794|67.13702127659575| 0.2765957446808511| 0.1347517730496454|    124.4143

In [10]:
df.groupBy('stroke').count().show()

+------+-----+
|stroke|count|
+------+-----+
|     1|  249|
|     0| 4861|
+------+-----+



In [11]:
#influence of work type on getting stroke
#more private
df_stroke = df.groupBy('work_type', 'stroke').count().select('stroke','work_type',f.col('count').alias('work_type_count'))
df_stroke = df_stroke.filter(df['stroke'] == 1).select('work_type', 'work_type_count').orderBy(df_stroke['work_type_count'].desc())
df_stroke.show()

+-------------+---------------+
|    work_type|work_type_count|
+-------------+---------------+
|      Private|            149|
|Self-employed|             65|
|     Govt_job|             33|
|     children|              2|
+-------------+---------------+



In [12]:
#participated in this clinic measurement
#58% Female. 41% Male
df_mes = df.groupBy('gender').count().select('gender',f.col('count').alias('count_gender')).withColumn('percentage_of_gender',(f.col('count_gender') / df.count()) * 100 ).select('gender','count_gender',(format_number('percentage_of_gender',2).alias('percentage_of_gender')))

df_mes.show()

+------+------------+--------------------+
|gender|count_gender|percentage_of_gender|
+------+------------+--------------------+
|Female|        2994|               58.59|
| Other|           1|                0.02|
|  Male|        2115|               41.39|
+------+------------+--------------------+



In [13]:
# how many female/male have a stroke
# 2,11% Male. 2,76% Female
df_gen_str_M = df.groupBy('gender', 'stroke').count().select('gender',f.col('count').alias('count_gender')).withColumn('percentage_of_stroke',(f.col('count_gender') / df.count()) * 100 ).filter((df['stroke'] == 1) & (df['gender'] == 'Male')).select('gender','count_gender',(format_number('percentage_of_stroke',2).alias('percentage_of_stroke')))

df_gen_str_M.show()

df_gen_str_F = df.groupBy('gender', 'stroke').count().select('gender',f.col('count').alias('count_gender')).withColumn('percentage_of_stroke',(f.col('count_gender') / df.count()) * 100 ).filter((df['stroke'] == 1) & (df['gender'] == 'Female')).select('gender','count_gender',(format_number('percentage_of_stroke',2).alias('percentage_of_stroke')))

df_gen_str_F.show()

+------+------------+--------------------+
|gender|count_gender|percentage_of_stroke|
+------+------------+--------------------+
|  Male|         108|                2.11|
+------+------------+--------------------+

+------+------------+--------------------+
|gender|count_gender|percentage_of_stroke|
+------+------------+--------------------+
|Female|         141|                2.76|
+------+------------+--------------------+



In [22]:
#sort by age that have a stroke
df_ag = df.groupBy('age','stroke').count().filter(df['stroke'] == 1).select('age', 'count')
df_ag.show()

+----+-----+
| age|count|
+----+-----+
|82.0|    9|
|51.0|    4|
|71.0|    7|
|69.0|    6|
|65.0|    3|
|78.0|   21|
|47.0|    1|
|61.0|    6|
|60.0|    4|
|70.0|    6|
|54.0|    6|
|56.0|    3|
|63.0|    9|
|52.0|    2|
|58.0|    7|
|55.0|    2|
|14.0|    1|
|68.0|    9|
|74.0|    9|
|59.0|    8|
+----+-----+
only showing top 20 rows



In [16]:
#sort by stroke and age > 50
df_age = df.groupBy('age','stroke').count().select('stroke','age',f.col('count').alias('age_count')).withColumn('percentage_of_stroke',(f.col('age_count') / df.count()) * 100 ).filter(df['stroke'] == 1).select('age','age_count',(format_number('percentage_of_stroke',2).alias('percentage_of_stroke')))

df_age = df_age.orderBy(df_age['age_count'].desc())

df_age.show()

+----+---------+--------------------+
| age|age_count|percentage_of_stroke|
+----+---------+--------------------+
|78.0|       21|                0.41|
|79.0|       17|                0.33|
|80.0|       17|                0.33|
|81.0|       14|                0.27|
|57.0|       11|                0.22|
|76.0|       10|                0.20|
|63.0|        9|                0.18|
|74.0|        9|                0.18|
|82.0|        9|                0.18|
|68.0|        9|                0.18|
|77.0|        8|                0.16|
|59.0|        8|                0.16|
|58.0|        7|                0.14|
|71.0|        7|                0.14|
|61.0|        6|                0.12|
|75.0|        6|                0.12|
|54.0|        6|                0.12|
|69.0|        6|                0.12|
|70.0|        6|                0.12|
|72.0|        6|                0.12|
+----+---------+--------------------+
only showing top 20 rows



In [19]:
# calculate the number of stroke cases for people after 50 years 
df.filter((df['stroke'] == 1) & (df['age'] > '50')).count()

226

In [20]:
#percentage person over 50 that have stroke
#4.4 %
df_age_per = ((df.filter((df['stroke'] == 1) & (df['age'] > '50')).count() ) / df.count() ) * 100 
df_age_per


4.422700587084149